In [13]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf # only for importing mnist dataset

In [ ]:
# Example
# Reading an image
img = cv2.imread('messi.jpeg',cv2.IMREAD_GRAYSCALE)/255
plt.imshow(img,cmap='gray')
img.shape

CONVOLUTION OPERATION

In [14]:
class convolution_operations:

    def __init__(self, num_filters, filter_size):
        self.num_filters = num_filters
        self.filter_size = filter_size
        self.conv_filter = np.random.randn(num_filters,filter_size,filter_size)/(filter_size*filter_size)

    def image_region(self, image):
        height , width = image.shape
        self.image = image
        for j in range(height - self.filter_size + 1):
            for k in range(width - self.filter_size + 1):
                image_patch = image[ j : (j+self.filter_size), k : (k+self.filter_size)]
                yield image_patch, j, k

    def forward_propogation(self, image):
        height , width = image.shape
        conv_out = np.zeros((height - self.filter_size + 1, width - self.filter_size + 1, self.num_filters))
        for image_patch, i, j in self.image_region(image):
            conv_out[i,j] = np.sum(image_patch*self.conv_filter, axis = (1,2))
        return conv_out
        
    def backward_propogation(self, dL_dout, learning_rate):
        dL_dF_params = np.zeros(self.conv_filter.shape)
        for image_patch, i, j in self.image_region(self.image):
            for k in range(self.num_filters):
                dL_dF_params[k] += image_patch*dL_dout[i,j,k]
        self.conv_filter -= learning_rate*dL_dF_params
        return dL_dF_params

In [ ]:
# Example to visualize the output of convolution operation
conn = convolution_operations(18,7)
out = conn.forward_propogation(img)
out.shape

In [ ]:
# plotting the example (output of a convolution operation)
plt.imshow(out[:,:,17], cmap='gray')
plt.show

MAX POOLING OPERATION

In [15]:
class Max_Pool:
    def __init__(self,filter_size):
        self.filter_size = filter_size

    def image_region(self, image):
        new_height = image.shape[0] // self.filter_size
        new_width = image.shape[1] // self.filter_size
        self.image = image

        for i in range(new_height):
            for j in range(new_width):
                image_patch = image[(i * self.filter_size) : (i * self.filter_size + self.filter_size), (j * self.filter_size) : (j * self.filter_size + self.filter_size)]
                yield image_patch, i, j

    def forward_propogation(self, image):
        height, width, num_filters = image.shape
        output = np.zeros((height // self.filter_size, width // self.filter_size, num_filters))

        for image_patch, i, j in self.image_region(image):
            output[i,j] = np.max(image_patch, axis=(0,1))

        return output
    
    def back_propogation(self, dL_dout):
        dL_dmax_pool = np.zeros(self.image.shape)
        for image_patch, i, j in self.image_region(self.image):
            height, width, num_filters = image_patch.shape
            maximum_val = np.amax(image_patch, axis = (0,1))

            for i1 in range(height):
                for j1 in range(width):
                    for k1 in range(num_filters):
                        if image_patch[i1,j1,k1] == maximum_val[k1]:
                            dL_dmax_pool[i*self.filter_size + i1, j*self.filter_size + j1, k1] = dL_dout[i, j, k1]
            
        return dL_dmax_pool



In [ ]:
# Example to visualize the output of a maxpooling operation after convolution (output of convolution is fed as input to maxpooling forward propag)
conn2 = Max_Pool(4)
out2 = conn2.forward_propogation(out)
out2.shape

In [ ]:
# plotting the example (output of a maxpooling operation)
plt.imshow(out2[:,:,17], cmap='gray')
plt.show

SOFTMAX OPERATION - ACTIVATION FUNCTION

In [16]:
class Softmax:
    
    def __init__(self, input_node, softmax_node):
        self.weight = np.random.randn(input_node,softmax_node)/input_node
        self.bias = np.zeros(softmax_node)

    def forward_propagation(self, image):
        
        self.orig_im_shape = image.shape # used in backpropagation
        image_modified = image.flatten()
        self.modified_input = image_modified # to be used in back propagtion
        output_val = np.dot(image_modified, self.weight) + self.bias
        self.out = output_val
        exp_out = np.exp(output_val)
        return exp_out/np.sum(exp_out, axis=0)
    
    def back_propagation(self, dL_dout, learning_rate):
        for i, grad in enumerate(dL_dout):
            if grad == 0:
                continue

            transformation_eq = np.exp(self.out)
            S_total = np.sum(transformation_eq)

            # Gradients with respect to out (z)
            dy_dz = -transformation_eq[i]*transformation_eq / (S_total**2)
            dy_dz[i] = transformation_eq[i]*(S_total - transformation_eq[i]) / (S_total**2)

            # Gradients of totals against weights/biases/input
            dz_dw = self.modified_input
            dz_db = 1
            dz_d_inp = self.weight

            # Gradients of loss against totals
            dL_dz = grad * dy_dz

            # Gradients of loss against weights/biases/input
            dL_dw = dz_dw[np.newaxis].T @ dL_dz[np.newaxis]
            dL_db = dL_dz * dz_db
            dL_d_inp = dz_d_inp @ dL_dz

        self.weight -= learning_rate * dL_dw
        self.bias -= learning_rate * dL_db

        return dL_d_inp.reshape(self.orig_im_shape)

In [ ]:
# Example is passed to softmax after maxpooling operation - Here it gets flattened into number of linear dimension (nodes or classes)
conn3 = Softmax(373*248*18,10)
out3 = conn3.forward_propagation(out2)
print(out3)

IMPLEMENTATION (TRAINING WITH ACTUAL DATA)

In [17]:
# Loading the mnist dataset from tensorflow
mnist = tf.keras.datasets.mnist

# Split dataset into training and testing data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
# Printing the shape of dataset for understanding
print(x_train.shape)
print(x_test.shape)

In [18]:
# Among the number of datas, we take 1500 datas for both training and testing
train_images = x_train[:1500]
train_labels = y_train[:1500]
test_images = x_test[:1500]
test_labels = y_test[:1500]

In [19]:
# creating instances of convolution, maxppoling and activation layers
conv = convolution_operations(8,3)
pool = Max_Pool(2)
softmax = Softmax(13*13*8,10)

In [20]:
def cnn_forward_prop(image,label):

    out_p = conv.forward_propogation((image/255)-0.5)
    out_p = pool.forward_propogation(out_p)
    out_p = softmax.forward_propagation(out_p)

    cross_ent_loss = -np.log(out_p[label])
    accuracy_eval = 1 if np.argmax(out_p) == label else 0

    return out_p, cross_ent_loss, accuracy_eval


In [21]:
def training_cnn(image, label, learn_rate=.005):

    # Forward
    out, loss, acc = cnn_forward_prop(image, label)

    # calculate initial gradient
    gradient = np.zeros(10)
    gradient[label] = -1 / out[label]

    # Backpropagation
    grad_back = softmax.back_propagation(gradient, learn_rate)
    grad_back = pool.back_propogation(grad_back)
    grad_back = conv.backward_propogation(grad_back, learn_rate)

    return loss, acc

In [22]:
for epoch in range(4):
    print(f"Epoch {epoch+1} --->")

    #shuffle the training data
    shuffle_data = np.random.permutation(len(train_images))
    train_images = train_images[shuffle_data]
    train_labels = train_labels[shuffle_data]

    #Training the cnn
    loss = 0
    num_correct = 0
    for i, (im, label) in enumerate(zip(train_images, train_labels)):
        if i % 100 == 0:
            print(f"{i+1} steps out of 100 steps: AVerage Loss {loss/100:.3f} and Accuracy: {num_correct}%")
            loss = 0
            num_correct = 0
        
        l1, accu = training_cnn(im,label)
        loss += l1
        num_correct += accu

print("**Testing Phase")
loss = 0
num_correct = 0
for im, label in zip(test_images, test_labels):
    _, l1, accu = cnn_forward_prop(im, label)
    loss += l1
    num_correct += accu

num_tests = len(test_images)
print('Test Loss : ', loss/num_tests)
print('Test Accuracy : ', num_correct/num_tests)

Epoch 1 --->
1 steps out of 100 steps: AVerage Loss 0.000 and Accuracy: 0%
101 steps out of 100 steps: AVerage Loss 2.246 and Accuracy: 16%
201 steps out of 100 steps: AVerage Loss 2.039 and Accuracy: 33%
301 steps out of 100 steps: AVerage Loss 1.611 and Accuracy: 51%
401 steps out of 100 steps: AVerage Loss 1.162 and Accuracy: 65%
501 steps out of 100 steps: AVerage Loss 1.072 and Accuracy: 71%
601 steps out of 100 steps: AVerage Loss 0.819 and Accuracy: 76%
701 steps out of 100 steps: AVerage Loss 0.723 and Accuracy: 74%
801 steps out of 100 steps: AVerage Loss 0.783 and Accuracy: 73%
901 steps out of 100 steps: AVerage Loss 0.613 and Accuracy: 84%
1001 steps out of 100 steps: AVerage Loss 0.679 and Accuracy: 75%
1101 steps out of 100 steps: AVerage Loss 0.726 and Accuracy: 76%
1201 steps out of 100 steps: AVerage Loss 0.518 and Accuracy: 82%
1301 steps out of 100 steps: AVerage Loss 0.461 and Accuracy: 86%
1401 steps out of 100 steps: AVerage Loss 0.582 and Accuracy: 83%
Epoch 2 --